<a href="https://colab.research.google.com/github/Viktory4121/MM/blob/main/%D0%9B%D0%A03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Лабораторная работа №3.
#### Клеточные конечные автоматы.
##### Вариант 11.

In [1]:
import numpy as np
import pandas as pd
import random
import copy
import math
import itertools as it


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

#### Входные параметры:

In [2]:
t = [i for i in range(1, 11)]
cell_auto0 = [[0 for j in range(10)] for i in range(10)]
cell_auto0[5][6] = 1
cell_auto0[9][2] = 1
cell_auto0[0][3] = 1
cell_auto0[4][4] = 1
cell_auto0[2][4] = 1
cell_auto0[3][0] = 1
cell_auto0[2][8] = 1
cell_auto0[3][5] = 1
cell_auto0[2][5] = 1
cell_auto0[3][8] = 1
cell_auto0 = pd.DataFrame(cell_auto0)
cell_auto0.columns = [i for i in range(10)]
cell_auto0.index = [i for i in range(10)]

#### Генерация основной части таблицы состояний:

In [12]:
def generation_table_states():
  table = pd.DataFrame(it.product([0,1], repeat=5))
  table[5] = [0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0]
  table.columns = [i for i in range(6)]
  table.index = [i for i in range(32)]
  return table

#### Функция проверки клетки на возбуждённое состояние:

In [13]:
def generate_cell_auto(table, avt1):
  result = [[0 for j in range(10)] for i in range(10)]

  for j in range(10):
    for k in range(10):
      #проверка на границы
      if (j == 0):
        c1 = 0
      else: c1 = avt1[j-1][k]
      if (k == 0):
        c3 = 0
      else: c3 = avt1[j][k-1]
      if (j == 9):
        c2 = 0
      else: c2 = avt1[j+1][k]
      if (k == 9):
        c4 = 0
      else: c4 = avt1[j][k+1]

      for i in range(32):
        if(table[i][0] == avt1[j][k] & table[i][1] == c1 & table[i][2] == c2 & table[i][3] == c3 & table[i][4] == c4):
          result[j][k] = 1
          break
  return result

Построение полинома Жегалкина:

In [54]:
table_states = generation_table_states()
t = np.array(table_states.copy())

In [43]:
#from sympy import *
#a = [0] * 33
#for i in range(33):
  #a[i].append(Symbol('a' + str(i)))

#rez1 = a[0] ^ (a[1] * t[i][0]) ^ (a[2] * t[i][1]) ^ (a[3] * t[i][2]) ^ (a[4] * t[i][3]) ^ (a[5] * t[i][4]) ^
  #(a[6] * t[i][0] * t[i][1]) ^ (a[7] * t[i][0] * t[i][2]) ^ (a[8] * t[i][0] * t[i][3]) ^ (a[9] * t[i][0] * t[i][4]) ^ 
  #(a[10] * t[i][1] * t[i][2]) ^ (a[11] * t[i][1] * t[i][3]) ^ (a[12] * t[i][1] * t[i][3]) ^ (a[13] * t[i][1] * t[i][4]) ^ 
  #(a[14] * t[i][2] * t[i][3]) ^ (a[15] * t[i][2] * t[i][4]) ^ (a[16] * t[i][3] * t[i][4]) ^
  #(a[17] * t[i][0] * t[i][1] * t[i][2]) ^ (a[18] * t[i][0] * t[i][1] * t[i][3]) ^ (a[19] * t[i][0] * t[i][1] * t[i][4]) ^ 
  #(a[20] * t[i][0] * t[i][2] * t[i][3]) ^ (a[21] * t[i][0] * t[i][2] * t[i][4]) ^ (a[22] * t[i][0] * t[i][3] * t[i][4]) ^ 
  #(a[23] * t[i][1] * t[i][2] * t[i][3]) ^ (a[24] * t[i][1] * t[i][2] * t[i][4]) ^ (a[25] * t[i][1] * t[i][3] * t[i][4]) ^ 
  #(a[26] * t[i][2] * t[i][3] * t[i][4]) ^ (a[27] * t[i][0] * t[i][1] * t[i][2] * t[i][3]) ^ (a[28] * t[i][0] * t[i][1] * t[i][2] * t[i][4]) ^ 
  #(a[29] * t[i][0] * t[i][1] * t[i][3] * t[i][4]) ^ (a[30] * t[i][0] * t[i][2] * t[i][3] * t[i][4]) ^ (a[31] * t[i][1] * t[i][2] * t[i][3] * t[i][4]) ^ 
  #(a[32] * t[i][0] * t[i][1] * t[i][2] * t[i][3] * t[i][4])
  

##### Полином Жегалкина имеет вид:
$$x=a_0 + a_1 x_1 + a_2 x_2 + a_3 x_3 + a_4 x_4 + a_5 x_5 + $$
$$a_{12} x_1 x_2 + a_{13} x_1 x_3 + a_{14} x_1 x_4 + a_{15} x_1 x_5 +
a_{23} x_2 x_3 + a_{24} x_2 x_4 + a_{25} x_2 x_5 +
a_{34} x_3 x_4 + a_{35} x_3 x_5 + a_{45} x_4 x_5 + $$
$$a_{123} x_1 x_2 x_3 + a_{124} x_1 x_2 x_4 + a_{125} x_1 x_2 x_5 +
a_{134} x_1 x_3 x_4 + a_{135} x_1 x_3 x_5 + a_{145} x_1 x_4 x_5 +
a_{234} x_2 x_3 x_4 + a_{235} x_2 x_3 x_5 + a_{245} x_2 x_4 x_5 +
a_{345} x_3 x_4 x_5 + $$
$$a_{1234} x_1 x_2 x_3 x_4 + a_{1235} x_1 x_2 x_3 x_5 +
a_{1245} x_1 x_2 x_4 x_5 + a_{1345} x_1 x_3 x_4 x_5 +
a_{2345} x_2 x_3 x_4 x_5 + a_{12345} x_1 x_2 x_3 x_4 x_5$$
##### где + - исключающее или, * - конъюнкция

In [119]:
#Рассчёт вручную:
print('Значения x1, x2, x3, x4, x5:')
print(t[0][0:5])
print('Полином будет иметь вид:')
print(str(t[0][5]) + ' = a0 + 0 + ... + 0')
print('a0 = ' + str(0))

print('\nЗначения x1, x2, x3, x4, x5:')
print(t[1][0:5])
print('Полином будет иметь вид:')
print(str(t[1][5]) + ' = a0 + a5 * x5 + 0 ... + 0')
print('a5 = ' + str(0))

print('\nЗначения x1, x2, x3, x4, x5:')
print(t[2][0:5])
print('Полином будет иметь вид:')
print(str(t[2][5]) + ' = a0 + a4 * x4 + 0 + 0 ... + 0')
print('a4 = ' + str(0))

print('\nЗначения x1, x2, x3, x4, x5:')
print(t[3][0:5])
print('Полином будет иметь вид:')
print(str(t[3][5]) + ' = a0 + a4 * x4 + a5 * x5 + a45 * x4 * x5 + 0 + 0 ... + 0')
print('a45 = ' + str(0))

print('\nЗначения x1, x2, x3, x4, x5:')
print(t[4][0:5])
print('Полином будет иметь вид:')
print(str(t[4][5]) + ' = a0 + x3 * a3 + 0 + 0 ... + 0')
print('a3 = ' + str(0))

print('\nЗначения x1, x2, x3, x4, x5:')
print(t[5][0:5])
print('Полином будет иметь вид:')
print(str(t[5][5]) + ' = a0 + a3 * x3 + a5 * x5 + a35 * x3 * x5 + 0 + ... + 0')
print('a35 = ' + str(0))

print('\nЗначения x1, x2, x3, x4, x5:')
print(t[6][0:5])
print('Полином будет иметь вид:')
print(str(t[6][5]) + ' = a0 + a3 * x3 + a4* x4 + a34 * x3 * x4 + 0 + ... + 0')
print('a34 = ' + str(0))

print('\nЗначения x1, x2, x3, x4, x5:')
print(t[7][0:5])
print('Полином будет иметь вид:')
print(str(t[7][5]) + ' = a0 + a3 * x3 + a4 * x4 + a5 * x5 + a34 * x3 * x4 + a45 * x4 * x5 + a35 * x3 * x5 + a345 * x3 * x4 * x5')
print('a345 = ' + str(0))

print('\nЗначения x1, x2, x3, x4, x5:')
print(t[8][0:5])
print('Полином будет иметь вид:')
print(str(t[8][5]) + ' = a2 * x2')
print('a2 = ' + str(0))

print('\nЗначения x1, x2, x3, x4, x5:')
print(t[9][0:5])
print('Полином будет иметь вид:')
print(str(t[9][5]) + ' = 0 + 0 + a25 * x2 * x5')
print('a25 = ' + str(0))

print('\nЗначения x1, x2, x3, x4, x5:')
print(t[10][0:5])
print('Полином будет иметь вид:')
print(str(t[10][5]) + ' = 0 + 0 + a24 * x2 * x4')
print('a24 = ' + str(0))

print('\nЗначения x1, x2, x3, x4, x5:')
print(t[11][0:5])
print('Полином будет иметь вид:')
print(str(t[11][5]) + ' = 0 + ... + a245 * x2 * x4 * x5')
print('a245 = ' + str(1))

print('\nЗначения x1, x2, x3, x4, x5:')
print(t[12][0:5])
print('Полином будет иметь вид:')
print(str(t[12][5]) + ' = 0 + 0 + 0 + a23 * x2 * x3')
print('a23 = ' + str(1))



Значения x1, x2, x3, x4, x5:
[0 0 0 0 0]
Полином будет иметь вид:
0 = a0 + 0 + ... + 0
a0 = 0

Значения x1, x2, x3, x4, x5:
[0 0 0 0 1]
Полином будет иметь вид:
0 = a0 + a5 * x5 + 0 ... + 0
a5 = 0

Значения x1, x2, x3, x4, x5:
[0 0 0 1 0]
Полином будет иметь вид:
0 = a0 + a4 * x4 + 0 + 0 ... + 0
a4 = 0

Значения x1, x2, x3, x4, x5:
[0 0 0 1 1]
Полином будет иметь вид:
0 = a0 + a4 * x4 + a5 * x5 + a45 * x4 * x5 + 0 + 0 ... + 0
a45 = 0

Значения x1, x2, x3, x4, x5:
[0 0 1 0 0]
Полином будет иметь вид:
0 = a0 + x3 * a3 + 0 + 0 ... + 0
a3 = 0

Значения x1, x2, x3, x4, x5:
[0 0 1 0 1]
Полином будет иметь вид:
0 = a0 + a3 * x3 + a5 * x5 + a35 * x3 * x5 + 0 + ... + 0
a35 = 0

Значения x1, x2, x3, x4, x5:
[0 0 1 1 0]
Полином будет иметь вид:
0 = a0 + a3 * x3 + a4* x4 + a34 * x3 * x4 + 0 + ... + 0
a34 = 0

Значения x1, x2, x3, x4, x5:
[0 0 1 1 1]
Полином будет иметь вид:
0 = a0 + a3 * x3 + a4 * x4 + a5 * x5 + a34 * x3 * x4 + a45 * x4 * x5 + a35 * x3 * x5 + a345 * x3 * x4 * x5
a345 = 0

Значения

In [120]:
print('\nЗначения x1, x2, x3, x4, x5:')
print(t[13][0:5])
print('Полином будет иметь вид:')
print(str(t[13][5]) + ' = 0 + 0 + 0 + a23 * x2 * x3')
print('a = ' + str(1))


Значения x1, x2, x3, x4, x5:
[0 1 1 0 1]
Полином будет иметь вид:
1 = 0 + 0 + 0 + a23 * x2 * x3
a = 1


#### Работа с таблицей состояний:

In [5]:
print('Таблица состояний для клеточного автомата:')
data = pd.DataFrame(table_states.copy())
data.columns = ['x(t*;c1;c2)', 'x(t*;c1-1;c2)', 'x(t*;c1+1;c2)', 'x(t*;c1;c2-1)', 'x(t*;c1;c2+1)', 'x(t;c1;c2)']
print(data)
print('-------------------------------------------------------------------')

print('\nКлеточный автомат в момент времени t = 0:')
print(cell_auto0)
print('-------------------------------------------------------------------')

table_old = cell_auto0
for i in t:
  print('Клеточный автомат в момент времени t = ' + str(i))
  table_new = generate_cell_auto(np.array(table_states), table_old)
  table_old = table_new
  print_table = pd.DataFrame(table_new)
  print(print_table)
  print('-------------------------------------------------------------------')

Таблица состояний для клеточного автомата:
    x(t*;c1;c2)  x(t*;c1-1;c2)  x(t*;c1+1;c2)  x(t*;c1;c2-1)  x(t*;c1;c2+1)  x(t;c1;c2)
0             0              0              0              0              0           0
1             0              0              0              0              1           0
2             0              0              0              1              0           0
3             0              0              0              1              1           0
4             0              0              1              0              0           0
5             0              0              1              0              1           0
6             0              0              1              1              0           0
7             0              0              1              1              1           0
8             0              1              0              0              0           0
9             0              1              0              0              1  